In [7]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import geopandas as gpd


data = gpd.read_file("./CaliFP.geojson", header=0)
data.info() #seeing what the data looks like


<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 16446 entries, 0 to 16445
Data columns (total 21 columns):
 #   Column          Non-Null Count  Dtype   
---  ------          --------------  -----   
 0   OBJECTID        16446 non-null  int64   
 1   YEAR_           16446 non-null  object  
 2   STATE           16446 non-null  object  
 3   AGENCY          16441 non-null  object  
 4   UNIT_ID         16429 non-null  object  
 5   FIRE_NAME       16435 non-null  object  
 6   INC_NUM         16110 non-null  object  
 7   ALARM_DATE      14821 non-null  object  
 8   CONT_DATE       8942 non-null   object  
 9   CAUSE           16422 non-null  float64 
 10  COMMENTS        14525 non-null  object  
 11  REPORT_AC       8478 non-null   float64 
 12  GIS_ACRES       16446 non-null  float64 
 13  C_METHOD        9101 non-null   float64 
 14  OBJECTIVE       16270 non-null  float64 
 15  FIRE_NUM        12620 non-null  object  
 16  COMPLEX_NAME    569 non-null    object  
 17  COMP

Convert geometry data to coordinates

In [8]:
data = data.set_crs(epsg = 3310 , allow_override=True)
data = data.to_crs(epsg = 4326)
data.geometry.bounds

,minx,miny,maxx,maxy
0,-120.001384,38.016715,-120.001384,38.016715
1,-120.001384,38.016715,-120.001384,38.016715
2,-120.001384,38.016715,-120.001384,38.016715
3,-120.001383,38.016716,-120.001383,38.016716
4,-120.001383,38.016721,-120.001383,38.016721
...,...,...,...,...
16441,-120.001403,38.016733,-120.001403,38.016733
16442,-120.001364,38.016703,-120.001363,38.016703
16443,-120.001393,38.016729,-120.001393,38.016729
16444,-120.001403,38.016733,-120.001403,38.016733


Remove fires not in california, before 1950, and less than 1000 acres

In [9]:
data.set_index('STATE' , inplace = True) #run entire cell only once after reading from above cell

print('entries:', len(data)) #print how many current entries data frame has
data.drop(index = 'NV' , axis = 0, inplace = True) 
data.drop(index = 'OR' , axis = 0, inplace = True) 
data.drop(index = 'AZ' , axis = 0 , inplace = True)
print('entries:', len(data))

# data = data[data['STATE'] != 'AZ'] alternate method of cleaning
data.reset_index('STATE' , inplace = True)

data = data.loc[data["YEAR_"] > "1949"] #loc function allows for sorting without reindexing
print('entries:', len(data))
data = data.loc[data['GIS_ACRES'] > 1000] #reassigns dataframe to the same dataframe w/ acres > 10
print('entries:', len(data))

entries: 16446
entries: 16369
entries: 16369
entries: 2889


All entries have year information

sort dataframe chronologically

In [10]:
data.set_index('YEAR_' , inplace = True)
data.sort_values(by = 'YEAR_' , ascending = True , inplace = True) #sort by year first, then sort by ALARM_DATE
data.reset_index('YEAR_' , inplace = True)

data['datetime'] = pd.to_datetime(data['ALARM_DATE'] , errors = 'coerce') #create new datetime column
data.set_index('datetime' , inplace = True) #set new column to index
data.sort_values(by='datetime', ascending = True, inplace = True) #sort values
data.reset_index('datetime' , inplace = True) #reset index to numbered list
data.drop('datetime' , axis = 1 , inplace = True) #delete the datetime column

print(len(data) , data.index)
data.tail

2889 RangeIndex(start=0, stop=2889, step=1)


<bound method NDFrame.tail of      YEAR_ STATE  OBJECTID AGENCY UNIT_ID      FIRE_NAME   INC_NUM  \
0     1950    CA     30496    CDF     BEU      ECHENIQUE  00000000   
1     1950    CA     22537    CDF     LMU  BAGGETT GULCH  00000000   
2     1950    CA     30378    CDF     MVU        BARRETT  00000000   
3     1950    CA     26332    CDF     BDU     NORTH FORK  00000000   
4     1950    CA     23893    CDF     RRU         RAWSON  00000000   
...    ...   ...       ...    ...     ...            ...       ...   
2884  2001    CA     40357    NPS     YNP         HOOVER  00001965   
2885  2003    CA     31308    USF     CNF                     None   
2886  2008    CA     38377    CDF     FKU        AVOCADO  00005639   
2887  2016    CA     34642    DOD     AFV         CANYON      None   
2888  2016    CA     34644    DOD     MCP         ROBLAR      None   

                     ALARM_DATE                  CONT_DATE  CAUSE  ...  \
0     1950-05-30T00:00:00+00:00                       N

We now want to drop the majority of columns in order to focus on location and time data.

In [11]:
columns_to_drop = ['STATE', 'OBJECTID',  'AGENCY', 'UNIT_ID',
       'INC_NUM',  'CONT_DATE', 'COMMENTS', 'REPORT_AC',
        'C_METHOD', 'OBJECTIVE', 'FIRE_NUM', 'COMPLEX_NAME',
       'COMPLEX_INCNUM', 'SHAPE_Length', 'SHAPE_Area']
data.drop(columns_to_drop , axis = 1 , inplace = True)
data.info()

<class 'geopandas.geodataframe.GeoDataFrame'>
RangeIndex: 2889 entries, 0 to 2888
Data columns (total 6 columns):
 #   Column      Non-Null Count  Dtype   
---  ------      --------------  -----   
 0   YEAR_       2889 non-null   object  
 1   FIRE_NAME   2889 non-null   object  
 2   ALARM_DATE  2651 non-null   object  
 3   CAUSE       2887 non-null   float64 
 4   GIS_ACRES   2889 non-null   float64 
 5   geometry    2889 non-null   geometry
dtypes: float64(2), geometry(1), object(3)
memory usage: 135.5+ KB


Export cleaned data to a .geojson

In [12]:
data.to_file('Clean_FP.geojson', driver = 'GeoJSON' , encoding = 'utf-8')